# Homework Assignment 1 -  Solutions to Differential Equations for a Baseball

## Python Starter

Importing needed modules into Jupyter notebook (make sure to install extentions on codespace!)

In [ ]:
import numpy as np #numpy
import matplotlib.pyplot as plt #matplotlib 

Defining functions

In [ ]:
def Quadratic(x,param):
    a,b,c = param
    return a*x**2+b*x+c

In [ ]:
Quadratic(7,[2,1,-10]) # use [] when defining multiple variables for one parameter 

Creating function data

In [ ]:
xvalues=np.arange(-3,3.1,0.1) # to generate list of equidistant numbers, must add a step to the desired end value
# np.linspace may have also been used, used to define number of steps rather that the step value

In [ ]:
yvalues=Quadratic(xvalues,[2,1,-10])

Plotting function

In [ ]:
#plot
fig, ax = plt.subplots()

ax.plot(xvalues, yvalues, linewidth=2.0) #defining plot parameters
plt.show()

## No Friction

### Analytical Solution

Solving $y\"(t)=-g$ using given analytical solution (used help from: https://github.com/Omjuice/OmjuicyElectrodynamics_python/blob/main/SolvingDiffrentialEquations.ipynb)

In [ ]:
def analytical_solution(t,g,v0,y0): #solution of y''(t) = -g
    return -0.5* g * t**2 + v0 * t + y0


Defining given values

In [ ]:
g = 9.81
h = 30 #initial height ball is being dropped from
v0_val = [-3,0,10]
t = np.linspace (0,5,20)
print (t)
y0 = 30

In [ ]:
for v0 in v0_val: #use for loop for known number of iterators
    y_values = analytical_solution (t, g, v0, h)
    plt.plot(t, y_values, label=f'Initial Velocity: {v0} m/s') #come back and define lowest y-val to be 0

#labeling plot
plt.xlabel('Time (s)')
plt.ylabel('Position (m)')
plt.legend()
plt.title('Position vs Time for Different Initial Velocities')
plt.show()

### Computational Solution

Tutorial used: https://www.youtube.com/watch?v=MM3cBamj1Ms&ab_channel=Mr.PSolver

In [ ]:
import scipy as sp
from scipy.integrate import odeint as od

In [ ]:
def dSdt(t,S): #defining a vector to solve coupled ODEs
    y, v = S #vector components -> check homework notes for converting second order to first order
    return[v, -9.81] #defining vector 

for v0 in v0_val:
    S0 = (y0, v0) #initial conditions of vector
    sol = od(dSdt, y0=S0, t=t, tfirst=True) #computational solution using odeint
    y_sol = sol.T[0] #extracts solutions for y component of S (y'(t) = v)
    plt.plot(t, y_sol, label=f'Initial Velocity(C): {v0} m/s')

for v0 in v0_val: #testing against analytical solution
    y_values = analytical_solution (t, g, v0, h)
    plt.scatter(t, y_values, label=f'Initial Velocity (A): {v0} m/s')

plt.xlabel('Time (s)')
plt.ylabel('Position (m)')
plt.legend()
plt.title('Position vs Time for Different Initial Velocities')
plt.show()

#success!

## With Friction

Solving $\"y=-mg-γ y'(t)$ using computational (used help from: https://github.com/Omjuice/OmjuicyElectrodynamics_python/blob/main/SolvingDiffrentialEquations.ipynb)

In [ ]:
g = 9.81
gamma = 0.009 # thanks Om! <- derived via Stokes Law
h = 30 #initial height ball is being dropped from
v0_val = [-3,0,10]
t = np.linspace (0,5,100)
m = 0.145
y0 = 30

In [ ]:
def dFdt(t,F): #defining a vector to solve coupled ODEs
    y, v = F #vector components -> check homework notes for converting second order to first order
    return[v, -g - (gamma/m) * v] #defining vector 
    #defined y0 and v0 earlier

def solwfriction(t, ysol): # from Om
    impact_indices = np.where(y_sol <= 0)[0]
    if len(impact_indices) == 0:
    # If the ball doesn't reach the ground, return None
        return None
    impact_index = impact_indices[0]
    # Interpolate to get a more accurate estimate of the impact time
    impact_time = np.interp(0, y_sol[impact_index-1:impact_index+1], t[impact_index-1:impact_index+1])
    return impact_time

for v0 in v0_val:
    F0 = (y0, v0) #initial conditions of vector
    newSol = od(dFdt, y0=F0, t=t, tfirst=True) #computational solution using odeint
    ysol = newSol.T[0] #extracts solutions for y component of S (y'(t) = v)
    print(solwfriction(t, ysol))
    plt.plot(t, ysol, label=f'Initial Velocity(C): {v0} m/s')

plt.xlabel('Time (s)')
plt.ylabel('Position (m)')
plt.legend()
plt.title('Position vs Time for Different Initial Velocities')
plt.show()